In [1]:
from Ks import *
import pickle as pkl

# load data
f = open("data.pickle", "rb")
data = pkl.load(f)

In [2]:
import numpy as np

features = []
labels = []
state = []

for i in data:
    tmp = [i.bigCategory, i.day, i.goal, i.backers]
    features.append(tmp)
    labels.append(i.pleged)
    state.append(i.state)

X = np.array(features)
y = np.array(labels)
y_state = np.array(state)

In [3]:
from sklearn.preprocessing import OneHotEncoder, normalize

# one hot encode category
category = X[:, [0]]
enc = OneHotEncoder(handle_unknown='ignore')
category = enc.fit_transform(category).toarray()

In [4]:
# max normalize numeric features
d = X[:, [1, 3]]
d = normalize(d, axis=0, norm="max")

goal = X[:, 2].reshape(1, -1)
y = np.concatenate((y, np.squeeze(goal)), axis=0)

y = normalize(y.reshape(-1, 1), axis=0, norm="max").reshape(1, -1)
y = np.squeeze(y)

mid = int(len(y) / 2)
goal = y[mid:]
y = y[:mid]

In [5]:
X = np.concatenate((category, d, goal.reshape(-1, 1)), axis=1)

In [6]:
from sklearn.model_selection import KFold, train_test_split, StratifiedShuffleSplit

# K-fold cross validation
kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(X)

s = StratifiedShuffleSplit(test_size=0.2, random_state=11)

In [7]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

best_acc = 0
best_k = 0
predictMethod = ""

def saveModel(neigh):
    clf = open("KNN_clf.pkl", "wb")
    pkl.dump(neigh, clf)
    clf.close()

# use odd K to avoid tie
# find best number of neighbor
for i in range(11, 20, 2):

    for j, [train_index, test_index] in enumerate(s.split(X, y_state)):
    # for j, [train_index, test_index] in enumerate(kf.split(X)):
        X_train = X[train_index, :]
        y_train = y[train_index]
        y_train_state = y_state[train_index]

        X_test = X[test_index, :]
        y_test = y[test_index]
        y_test_state = y_state[test_index]

        # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        print("Building on K={} fold={}".format(i, j))
        neigh = NearestNeighbors(n_neighbors=i)
        neigh.fit(X_train)

        print("Testing...")
        n = neigh.kneighbors(X_test, i, return_distance=False)
        pledge_predict = []
        state_predict = []
        state_predict_plege = []

        for inx, nbrs in enumerate(n):
            nbrPlege = y_train[nbrs]
            p = np.mean(nbrPlege)
            pledge_predict.append(p)         
            goal = X_test[inx][-1]

            # goal = X_test[inx][2]
            # state = X_test[inx][2] >= y_test[inx]
            # actual_state.append(state)

            if p >= goal:
                state_predict_plege.append(True)
            else:
                state_predict_plege.append(False)

            nbrState = y_train_state[nbrs]
            nbrSuc = len(list(filter(lambda x: x, nbrState)))

            if nbrSuc > i / 2:
                state_predict.append(True)
            else:
                state_predict.append(False)
        
        RMSE = np.sqrt((np.square(pledge_predict - y_test).mean(axis=0)))
        tn, fp, fn, tp = confusion_matrix(y_test_state, state_predict).ravel()
        acc1 = accuracy_score(y_test_state, state_predict)
        acc2 = accuracy_score(y_test_state, state_predict_plege)
        
        print("           Positive       Negative")
        print("Positive     {}             {}".format(tp, fp))
        print("Negative     {}             {}".format(fn, tn))

        print(classification_report(y_test_state, state_predict))
        print("Accuracy predicted by neighbor state: {}".format(acc1))
        print("Accuracy predicted by neighbor pleged: {}".format(acc2))
        print("RMSE: {}".format(RMSE))

        if acc1 > best_acc:
            # save best classifier
            saveModel(neigh)
            best_acc = acc1
            predictMethod = "neighbor's states"
        
        if acc2 > best_acc:
            saveModel(neigh)
            best_acc = acc2
            predictMethod = "neighbor's plege"

Building on K=11 fold=0
Testing...
